In [1]:
import ast
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
%matplotlib inline

from minisom import MiniSom
from sklearn.preprocessing import minmax_scale, scale

In [2]:
data = pd.read_csv('../data_local/tracks_with_genres.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531191 entries, 0 to 531190
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        531191 non-null  int64  
 1   id                531191 non-null  object 
 2   name              531191 non-null  object 
 3   popularity        531191 non-null  int64  
 4   duration_ms       531191 non-null  int64  
 5   explicit          531191 non-null  int64  
 6   artists           531191 non-null  object 
 7   id_artists        531191 non-null  object 
 8   release_date      531191 non-null  object 
 9   danceability      531191 non-null  float64
 10  energy            531191 non-null  float64
 11  key               531191 non-null  int64  
 12  loudness          531191 non-null  float64
 13  mode              531191 non-null  int64  
 14  speechiness       531191 non-null  float64
 15  acousticness      531191 non-null  float64
 16  instrumentalness  53

In [3]:
# Convert columns containing lists to actual Python lists
keylist = ['rock',
 'filmi',
 'adult standards',
 'classical',
 'classic rock',
 'mellow gold',
 'classical performance',
 'latin',
 'soft rock',
 'album rock',
 'sufi',
 'folk rock',
 'vocal jazz',
 'classic bollywood',
 'desi pop']

data['genres'] = data['genres'].apply(ast.literal_eval)
explode_genres = data.explode('genres').dropna()
#explode_genres.head()
explode_genres_top15 = explode_genres['genres'].isin(keylist)
explode_genres_top15 = explode_genres[explode_genres_top15]

In [4]:
genres = explode_genres_top15['genres'].astype('category')
explode_genres_top15['genres'] = genres.cat.codes

<ipython-input-4-2bd4e3ec4157>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  explode_genres_top15['genres'] = genres.cat.codes


In [7]:
explode_genres_top15.describe()

,Unnamed: 0,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,genres
count,307056.000000,307056.000000,3.070560e+05,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000,307056.000000
mean,235258.067519,26.674942,2.628229e+05,0.008119,0.495408,0.479254,5.172232,-11.951838,0.706536,0.061807,0.550709,0.164069,0.219642,0.526301,115.876556,3.848106,6.768059
std,176738.741862,18.145138,1.680876e+05,0.089739,0.168515,0.263519,3.448971,5.683276,0.455350,0.066655,0.362161,0.310454,0.194897,0.267046,29.848654,0.489174,4.271504
min,2.000000,0.000000,3.344000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,66263.750000,11.000000,1.844670e+05,0.000000,0.375000,0.259000,2.000000,-14.778000,0.000000,0.033900,0.178000,0.000003,0.099000,0.309000,92.718000,4.000000,3.000000
50%,184790.000000,26.000000,2.329070e+05,0.000000,0.498000,0.475000,5.000000,-11.011000,1.000000,0.041800,0.616000,0.000490,0.140000,0.545000,113.657000,4.000000,7.000000
75%,442011.000000,39.000000,3.015870e+05,0.000000,0.616000,0.694000,8.000000,-7.900000,1.000000,0.059500,0.914000,0.095100,0.278000,0.751000,134.338000,4.000000,11.000000
max,531091.000000,94.000000,4.581483e+06,1.000000,0.986000,1.000000,11.000000,3.744000,1.000000,0.964000,0.996000,0.994000,1.000000,0.995000,243.372000,5.000000,14.000000



## Plotbars x axis is genres, y is features

In [12]:
feature_select = ['speechiness', 'valence', 'tempo', 'danceability']

#explode_genres_top15.select([feature_select])